## The forward and backward passes

In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations version

### Basic architecture

In [2]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [3]:
# num hidden
nh = 50

In [4]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [5]:
def lin(x, w, b): return x@w + b

In [9]:
x_valid.shape

torch.Size([10000, 784])

In [10]:
w1.shape

torch.Size([784, 50])

In [11]:
b1.shape

torch.Size([50])

In [16]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [17]:
def relu(x): return x.clamp_min(0.)

In [18]:
t

tensor([[ -0.09,  11.87, -11.39,  ...,   5.48,   2.14,  15.30],
        [  5.38,  10.21, -14.49,  ...,   0.88,   0.08,  20.23],
        [  3.31,   0.12,   3.10,  ...,  16.89,  -6.05,  24.74],
        ...,
        [  4.01,  10.35, -11.25,  ...,   0.23,  -5.30,  18.28],
        [ 10.62,  -4.27,  10.72,  ...,  -2.87,  -2.87,  18.23],
        [  2.84,  -0.22,   1.43,  ...,  -3.91,   5.75,   2.12]])

In [19]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [20]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [21]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [23]:
res

tensor([[  25.75],
        [ -13.06],
        [-114.79],
        ...,
        [ -67.44],
        [ -74.48],
        [ -60.19]])

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [24]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [25]:
(res-y_valid).shape

torch.Size([10000, 10000])

We need to get rid of that trailing (,1), in order to use `mse`.

In [26]:
res[:,0].shape

torch.Size([10000])

In [27]:
res.squeeze().shape

torch.Size([10000])

In [28]:
(res[:,0]-y_valid).shape

torch.Size([10000])

In [29]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [30]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [31]:
mse(preds, y_train)

tensor(4308.76)

### Gradients and backward pass

In [41]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [42]:
diff(3*x**2+9, x)

6*x

In [43]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [44]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [45]:
forward_and_backward(x_train, y_train)

In [46]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

这段代码似乎是某个神经网络或其他机器学习模型的一部分。让我解释一下每个部分的作用：

def get_grad(x): return x.g.clone(): 这个函数 get_grad 用于从张量 x 中提取梯度。它似乎设计用于与一个计算反向传播过程中梯度的框架或库一起使用。它返回梯度张量 x.g 的克隆，确保原始张量保持不变。

chks = w1, w2, b1, b2, x_train: 这行代码创建了一个元组 chks，其中包含变量 w1、w2、b1、b2 和 x_train。这些变量很可能表示权重（w1、w2）、偏置（b1、b2）和训练数据（x_train）。

grads = w1g, w2g, b1g, b2g, ig = tuple(map(get_grad, chks)): 这行代码通过调用 get_grad 函数从 chks 中获取梯度，并将结果存储在 grads 变量中。这可能是为了在后续的代码中对梯度进行进一步处理或分析。




map() 函数是 Python 中的一个内置函数，它的作用是对一个可迭代对象（比如列表、元组等）中的每个元素应用一个指定的函数，然后返回一个包含所有函数返回值的迭代器。

具体来说，map() 函数接受两个参数：一个函数和一个可迭代对象。它会将这个函数应用到可迭代对象的每个元素上，并返回一个新的迭代器，该迭代器包含了每个元素应用函数后的结果。

在给定的代码中，map(get_grad, chks) 就是把 get_grad 函数应用到 chks 这个元组中的每个元素上，然后返回一个包含了每个元素应用函数后的结果的迭代器。

We cheat a little bit and use PyTorch autograd to check our results.

In [47]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [48]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [49]:
loss = forward(xt2, y_train)
loss.backward()

In [50]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

## Refactor model

### Layers as classes

In [53]:
class A:
    def __call__(self,x): print(f'hi {x}')

In [55]:
a=A()
a("mm")

hi mm


In [56]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

这段代码定义了一个名为 Relu 的类，用来实现 ReLU（Rectified Linear Unit）激活函数的功能。让我解释一下这个类的作用和每个方法的功能：

__call__(self, inp): 这是一个特殊方法，当实例被调用时自动执行。这里的 __call__ 方法实现了 ReLU 函数的前向传播。它接受输入 inp，然后将其保存在 self.inp 中，并计算 ReLU 激活函数的输出，将结果保存在 self.out 中。ReLU 激活函数的定义是将输入值小于 0 的部分置为 0，因此这里使用了 clamp_min(0.) 方法来实现这一功能。最后，函数返回输出结果 self.out。

backward(self): 这个方法用于执行反向传播。在神经网络中，反向传播用于计算损失函数相对于每个参数的梯度，以便更新参数。在这个方法中，首先通过 (self.inp>0).float() 计算输入大于 0 的部分的导数，然后将其与输出的梯度相乘，得到输入的梯度。这里使用了一个常见的技巧，即在前向传播时保留了输入的副本，以便在反向传播时使用。

这个类的功能是实现了 ReLU 激活函数的前向传播和反向传播。

In [57]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

这段代码定义了一个名为 Lin 的类，它实现了一个线性层（全连接层）的功能。下面是这个类的功能和每个方法的作用：

__init__(self, w, b): 这是 Lin 类的构造函数。它接受两个参数 w 和 b，分别表示该线性层的权重和偏置。在构造函数中，将这两个参数保存在实例的属性 self.w 和 self.b 中。

__call__(self, inp): 这是一个特殊方法，用于使实例可调用。在这个方法中，接受输入 inp，将其保存在 self.inp 中，并将输入与权重 self.w 以及偏置 self.b 进行线性组合得到输出 self.out。这里调用了一个名为 lin 的函数来实现线性变换。

backward(self): 这个方法用于执行反向传播。在神经网络中，反向传播用于计算损失函数相对于每个参数的梯度，以便更新参数。在这个方法中，首先计算输入的梯度，然后计算权重和偏置的梯度。其中，self.inp.g 表示输入的梯度，self.w.g 表示权重的梯度，self.b.g 表示偏置的梯度。这里使用了一些张量运算来计算梯度，包括矩阵乘法 (@)、转置 (t())、以及对矩阵的求和 (sum(0))。

In [58]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

这段代码定义了一个名为 Mse 的类，用于计算均方误差（Mean Squared Error，MSE）的损失。让我解释一下这个类的功能和每个方法的作用：

__call__(self, inp, targ): 这是一个特殊方法，使得实例可以像函数一样被调用。在这个方法中，接受输入 inp 和目标 targ，并将它们保存在实例的属性 self.inp 和 self.targ 中。然后调用一个名为 mse 的函数计算输入和目标之间的均方误差，并将结果保存在 self.out 中，最后返回这个结果。

backward(self): 这个方法用于执行反向传播。在神经网络中，反向传播用于计算损失函数相对于每个参数的梯度，以便更新参数。在这个方法中，首先计算输入的梯度。对于均方误差损失函数，其导数关于输入的梯度可以通过简单的公式计算得到。然后将计算得到的梯度保存在 self.inp.g 中。

这个类的功能是实现了均方误差损失函数的前向传播和反向传播。

In [59]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [60]:
model = Model(w1, b1, w2, b2)

In [61]:
loss = model(x_train, y_train)

In [62]:
model.backward()

In [63]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [64]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

这个 Module 类定义了一个模块的基本结构，它是一个抽象类，用于构建更复杂的神经网络模型。让我解释一下这个类的结构和每个方法的作用：

__call__(self, *args): 这是一个特殊方法，使得实例可以像函数一样被调用。它接受任意数量的参数 args，将这些参数保存在 self.args 中，并调用 forward 方法来执行前向传播。forward 方法需要在子类中实现。

forward(self): 这个方法用于执行前向传播，计算模块的输出。由于这是一个抽象类，因此这个方法本身没有实现，而是通过 raise Exception('not implemented') 抛出了一个异常，提示子类需要实现这个方法。

backward(self): 这个方法用于执行反向传播。它调用了 bwd 方法，传递了模块的输出 self.out 和之前保存的参数 self.args。bwd 方法需要在子类中实现。

bwd(self): 这个方法用于实现具体的反向传播操作。和 forward 方法一样，这个方法也是一个抽象方法，通过 raise Exception('not implemented') 抛出了一个异常，提示子类需要实现这个方法。

这个 Module 类提供了一个模块的基本结构，定义了前向传播和反向传播的接口，但实际的实现需要在子类中完成。

In [65]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [66]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [67]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [68]:
model = Model(w1, b1, w2, b2)

In [69]:
loss = model(x_train, y_train)

In [70]:
model.backward()

In [71]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [72]:
from torch import nn
import torch.nn.functional as F

In [73]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [74]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [75]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [76]:
l0 = model.layers[0]
l0.b.grad